In [ ]:
%matplotlib inline
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
import pandas as pd
import numpy as np

## 读写文本格式的数据

In [ ]:
!cat pydata-book/ch06/ex1.csv

In [ ]:
df = pd.read_csv('pydata-book/ch06/ex1.csv')
df

In [ ]:
pd.read_table('pydata-book/ch06/ex1.csv', sep=',')

In [ ]:
!cat pydata-book/ch06/ex2.csv

In [ ]:
pd.read_csv('pydata-book/ch06/ex2.csv', header=None)

In [ ]:
pd.read_csv('pydata-book/ch06/ex2.csv', names=['a', 'b', 'c', 'd', 'message'])

In [ ]:
names = ['a', 'b', 'c', 'd', 'message']
pd.read_csv('pydata-book/ch06/ex2.csv', names=names, index_col='message')

In [ ]:
!cat pydata-book/ch06/csv_mindex.csv

In [ ]:
parsed = pd.read_csv('pydata-book/ch06/csv_mindex.csv', index_col=['key1', 'key2'])
parsed

In [ ]:
list(open('pydata-book/ch06/ex3.txt'))

In [ ]:
result = pd.read_table('pydata-book/ch06/ex3.txt', sep='\s+')
result    # 列名比数据行少，推断第一列应该是index

In [ ]:
!cat pydata-book/ch06/ex4.csv

In [ ]:
pd.read_csv('pydata-book/ch06/ex4.csv', skiprows=[0, 2, 3])

In [ ]:
!cat pydata-book/ch06/ex5.csv

In [ ]:
result = pd.read_csv('pydata-book/ch06/ex5.csv')
result
pd.isnull(result)

In [ ]:
result = pd.read_csv('pydata-book/ch06/ex5.csv', na_values=['NULL'])
result

In [ ]:
sentinels = {'message': ['foo', 'NA'], 'something': ['two']}
pd.read_csv('pydata-book/ch06/ex5.csv', na_values=sentinels)

### 逐块读取文本文件

In [ ]:
result = pd.read_csv('pydata-book/ch06/ex6.csv')
result

In [ ]:
pd.read_csv('pydata-book/ch06/ex6.csv', nrows=5)

In [ ]:
chunker = pd.read_csv('pydata-book/ch06/ex6.csv', chunksize=1000)
chunker

In [ ]:
from pandas import Series, DataFrame

In [ ]:
tot = Series([])
for piece in chunker:
    tot = tot.add(piece['key'].value_counts(), fill_value=0)

# tot = tot.sort_values(ascending=False)
# tot[:10]

### 将数据写出到文本格式

In [ ]:
!cat pydata-book/ch06/ex5.csv
data = pd.read_csv('pydata-book/ch06/ex5.csv')
data

In [ ]:
data.to_csv('test_out.csv')
!cat test_out.csv
!rm test_out.csv

In [ ]:
import sys
data.to_csv(sys.stdout, sep='|')

In [ ]:
data.to_csv(sys.stdout, na_rep='NULL')

In [ ]:
data.to_csv(sys.stdout, index=False, header=False, na_rep='NULL')

In [ ]:
data.to_csv(sys.stdout, index=False, columns=list('abc'), na_rep='NULL')

In [ ]:
dates = pd.date_range('1/1/2000', periods=7)
dates
ts = Series(np.arange(7), index=dates)
ts.to_csv('test_out.csv')
!cat test_out.csv
!rm test_out.csv

In [ ]:
Series.from_csv('pydata-book/ch06/tseries.csv', parse_dates=True)

### 手工处理分隔符格式

In [ ]:
!cat pydata-book/ch06/ex7.csv

In [ ]:
import csv
f = open('pydata-book/ch06/ex7.csv')
reader = csv.reader(f)
for line in reader:
    print line

In [ ]:
lines = list(csv.reader(open('pydata-book/ch06/ex7.csv')))
header, values = lines[0], lines[1:]
data_dict = {h: v for h, v in zip(header, zip(*values))}
data_dict

### JSON数据

### XML和HTML：Web信息收集

In [ ]:
from lxml import html
parsed = html.parse(open('pydata-book/ch06/fdic_failed_bank_list.html'))
doc = parsed.getroot()
links = doc.findall('.//a')
links[15:20]

In [ ]:
links[0].get('href')

In [ ]:
links[0].text_content()

In [ ]:
urls = [lnk.get('href') for lnk in doc.findall('.//a')]
urls[-10:]

In [ ]:
from lxml import objectify
path = 'pydata-book/ch06/mta_perf/Performance_MNR.xml'
parsed = objectify.parse(open(path))
root = parsed.getroot()

data = []
skip_fields = ['PARENT_SEQ', 'INDICATOR_SEQ', 'DESIRED_CHANGE', 'DECIMAL_PLACES']
for elt in root.INDICATOR:
    el_data = {}
    for child in elt.getchildren():
        if child.tag in skip_fields:
            continue
        el_data[child.tag] = child.pyval
    data.append(el_data)
    
perf = DataFrame(data)
perf

## 二进制数据格式

In [ ]:
frame = pd.read_csv('pydata-book/ch06/ex1.csv')
frame

### 使用HDF5格式

In [ ]:
store = pd.HDFStore('mydata.h5')
store['obj1'] = frame
store['obj1_col'] = frame['a']
store

### 读取Excel文件

In [ ]:
xls_file = pd.ExcelFile('pydata-book/ch06/ex1.xlsx')
table = xls_file.parse('Sheet1')

## 使用HTML和Web API

## 使用数据库

In [ ]:
import sqlite3

query = """
CREATE TABLE test (a VARCHAR(20), b VARCHAR(20), c REAL, d INTEGER);
"""
con = sqlite3.connect(':memory:')
con.execute(query)
con.commit()

In [ ]:
data = [('Atlanta', 'Georgia', 1.25, 5), 
       ('Tallahassee', 'Florida', 2.6, 3),
       ('Sacramento', 'California', 1.7, 5)]
stmt = 'INSERT INTO test VALUES (?, ?, ?, ?)'
con.executemany(stmt, data)
con.commit()

In [ ]:
cursor = con.execute('select * from test')
rows = cursor.fetchall()
rows

In [ ]:
cursor.description
zip(*cursor.description)

In [ ]:
DataFrame(rows, columns=zip(*cursor.description)[0])

In [ ]:
import pandas.io.sql as sql
sql.read_frame('select * from test', con)